In [85]:
# Income Fluctuation Problem
# Using Time Iteration and Endogenous Grid Method

# V(a,y) = max u(c)+βE[V(a',y')|y]
# s.t. c + a' = (1+r)a + y
# c>=0, a'>= lb
# y'\y ~ N
# Euler: u'(c(a,y)) = β(1+r)E[u'(c(a',y'))]     (> if a'=0)

# Time Iteration: from r, c(a',y'), β, P get c(a,y)
# c(a,y) = invup  β(1+r) dot(P,u'(c(a',y')))



In [86]:
# Shock Process Discretization
using Distributions
function tauchen(μ,σ,ρ,N)
    σ_ = σ/(1-ρ^2)^0.5    
    zu = μ + 3*σ
    zl = μ - 3*σ
    Z = collect(range(zl, zu, N))
    M = (Z[1:end-1]+Z[2:end])/2
    z = Normal(0, 1)
    P = zeros(N,N)
    for i in 2:N-1
        for j in 2:N-1
            P[i,j] = cdf(z, (M[j]-(1-ρ)*μ-ρ*Z[i])/σ)-cdf(z, (M[j-1]-(1-ρ)*μ-ρ*Z[i])/σ)
        end
    end

    P[:,1] = [cdf(z, (M[1]-(1-ρ)*μ-ρ*z_)/σ) for z_ in Z]
    P[:,end] = [(1-cdf(z, (M[end-1]-(1-ρ)*μ-ρ*z_)/σ)) for z_ in Z]
    return Z, P
    end

tauchen (generic function with 1 method)

In [92]:
# Parameters
ub = 200 # upper bound for assets
lb = 0 # lower bound for assets (can be negative)
β = 0.98 # patience
α = 0.11 # share of capital in income
δ = 0.025 # depreciation
γ = 0.5 # elasticity of substitution
r = 0.05 # interest rate

# Utility Function Tools
u(c, γ) = c.^(1-γ)/(1-γ) # period utility
up(c, γ) = c.^-γ # marginal utility
invup(up, γ) = up.^(-1\γ) # inverse of marginal utility

# Income Process
ρ = 0.9
N_Y = 5
σ = 0.13
μ = 0
Y, P = tauchen(μ,σ,ρ,N_Y) # Shock Space and Transition Matrix

# Asset and Income Grids
N_A = 10

function getGrids(ub, lb, N_A, N_Z, Y)
    Agrid = [i for i in range(lb, ub, N_A), j in 1:N_Y]
    Ygrid = [i for i = 1:N_A, j in Y]
    return Agrid, Ygrid
end

Agrid, Ygrid = getGrids(ub, lb, N_A, N_Z, Y)
size(Agrid), size(Ygrid)

((10, 5), (10, 5))

In [103]:
using LinearAlgebra
function TimeIteration(γ, β, P, r, C)
    # Input: C_future (N_A x N_Y matrix), P (N_Y x N_Y) and constants γ, β,r,
    # Output: C_now (N_A x N_Y matrix) 
    
    Vprime = (1+r).*up(C, γ) # V'(a',z') = (1+r)u'(c(a',z')
    RHS = β .* Vprime * P' # βE(V'(a,z))
    c = invup(RHS, γ) # u'^-1 (βE(V'(a,z)))
    return c # c(a,z) vector
end 

C = Agrid

TimeIteration(γ, β, P, r, C)

10×5 Matrix{Float64}:
 NaN        0.0      0.0      0.0      NaN
   2.60725  2.12167  1.94226  1.72382    2.16829
   3.10057  2.52311  2.30975  2.04998    2.57855
   3.43134  2.79228  2.55616  2.26867    2.85363
   3.68721  3.0005   2.74677  2.43785    3.06643
   3.89875  3.17264  2.90436  2.57771    3.24235
   4.08057  3.3206   3.0398   2.69792    3.39356
   4.2409   3.45106  3.15924  2.80392    3.52689
   4.38486  3.56821  3.26648  2.8991     3.64662
   4.5159   3.67484  3.3641   2.98574    3.75559

In [105]:
# Get "a" matrix from matrices a', z', c; and r
function getAssets(Agrid, Ygrid, r, C)
    # Input: a' (N_A x N_Y matrix), y' (N_A x N_Y matrix), c (N_A x N_Y matrix), r 
    # Output: a (N_A x N_Y matrix)
    # (1+r)a + y = c + a' ==> a = 1/(1+r) * (c + a' - y)
    return 1/(1+r) .* (C .+ Agrid .- Ygrid)
end

C = Agrid
getAssets(Agrid, Ygrid, r, C)

10×5 Matrix{Float64}:
  -0.952381   -0.952381   -0.952381   -0.952381   -0.952381
  40.4233     40.4233     40.4233     40.4233     40.4233
  81.7989     81.7989     81.7989     81.7989     81.7989
 123.175     123.175     123.175     123.175     123.175
 164.55      164.55      164.55      164.55      164.55
 205.926     205.926     205.926     205.926     205.926
 247.302     247.302     247.302     247.302     247.302
 288.677     288.677     288.677     288.677     288.677
 330.053     330.053     330.053     330.053     330.053
 371.429     371.429     371.429     371.429     371.429

In [71]:
# obtain aggregates

function getcapital(; policyfun, dist)
    K = reshape(policyfun, (length(dist),1))
    K = sum(K.*dist)
    return K
end

function AggregateConsumption(c,ϕ)
    C = reshape(c, (length(dist),1)) # flatten
    C = sum(C.*dist) 
    return C

function getconsumption(c, ϕ)
    C = reshape(cpolicy, (length(dist),1)) 
    C = sum(C.*dist)
    return C
end

(10,)(10, 10)

10-element Vector{Float64}:
 1.3759680890398716
 1.170700772119621
 1.26663643883576
 1.3613976559838235
 1.4416245055522952
 1.4973509406651988
 1.519284256250321
 1.512770434492169
 1.5090215959594018
 2.0134797261795554

In [ ]:
# Note: function takes both current and future interest rates as arguments.

function egm(hh; w, cnext, cbinding, r, rnext)
 """
    use endogenous grid method to obtain c_{t} and a_{t} given c_{t+1} 'cnext'
    
    #### Fields
    
    - 'hh': household tuple
    - 'w': wage rate
    - 'cnext': time t+1 consumption grid
    - 'cbinding': consumption grid when borrowing constraint binds
    - 'r': interest rate at time t
    - 'rnext': interest rate at time t+1
    
    #### Returns
    
    - 'c': time t consumption grid
    - 'anext': time t policy function
    
"""
    
@unpack gamma, beta, transition_matrix, Amat, Ymat, num_states = hh

# current policy functions on current grid
c = getc(gamma, beta, transition_matrix; rnext=rnext, cnext=cnext)
a = geta(Amat, Ymat; r=r, w=w, c=c)

cnonbinding = similar(Amat)
    
# get consumption policy function for current grid
for i = 1:num_states
    cnonbinding[:,i] = LinearInterpolation(a[:,i], c[:,i], extrapolation_bc = Line()).(Amat[:,i])
end

# update elements of consumption policy when borrowing constraint binds
# a[1,j] is the level of current assets that induces the borrowing constraint to bind exactly.
# Therefore, whenever current assets are below a[1,j], the borrowing constraint will be STRICTLY binding.
# Note that this uses the monotonicity of the policy rule.

for j = 1:num_states
   c[:,j] = (Amat[:,j] .> a[1,j]) .*cnonbinding[:,j] .+ (Amat[:,j] .<= a[1,j]).*cbinding[:,j]
end
    
# update saving policy function with new consumption function
anext = @. (1+r)*Amat + w*Ymat - c

return c, anext
end

In [ ]:
function EGM(
        # input: γ, β, P, Agrid, Ygrid, N_A, N_Z, r, 
        # output: 